**Define verifier class**

In [2]:
import random
from functools import reduce

class Verifier:
    
    def __init__(self):
        self.questions = [[0, 0, 0], [1, 1, 0], [1, 0, 1], [0, 1, 1]]
    
    # randomly sample one question
    def sample_question(self):
        return random.choice(self.questions)
    
    def query_alice(self, question):
        return question[0]
    
    def query_bob(self, question):
        return question[1]
    
    def query_charlie(self, question):
        return question[2]
    
    # check if the winning condition is satisfied.
    def judge(self, question, a, b, c):
        if a not in [0, 1] or b not in [0, 1] or c not in [0, 1]:
            return false
        
        # xor(answer) == or(question)
        return reduce((lambda x, y: (x + y) % 2), [a, b, c]) == reduce((lambda x, y: x | y), question)


**Define player class**

In [3]:
class Player:
    
    """
    initialize player with their shared resources,
    which will be typically a list of random bits.
    ex: shared_resources = [c_1, c_2]
    """
    def __init__(self, random_resource):
        self.r0 = random_resource[0]
        self.r1 = random_resource[1]
    
    """
    we will now define deterministic strategies:
    On one bit input, there are two possible deterministic strategies:
    output the same input bit, or output the opposite one.
    This can be captured by the following equation:
    a = x * r_0 \or r_1
    where (x, a) are the input and output for Alice, and r_0, r_1 are the random bits.
    """ 
    def strategy(self, question):
        return (question * self.r0 + self.r1) % 2
        
    """
    Player answer a given question with a predefined strategy
    which is a function that takes in the shared resources and 
    question bit and output an answer bit
    """
    def answer(self, question):
        return self.strategy(question)
    
# test run:
Alice = Player([0,0]) # Alice with a random bits r_0 = 0, r_1 = 0

print(Alice.answer(0)) # x = 0
print(Alice.answer(1)) # x = 1

0
0


In [4]:
# We will now enumerate all 64 possible joint deterministic strategies
def generate_randomness():
    single = [[r0, r1] for r0 in [0, 1] for r1 in [0, 1]]
    return [[x, y, z] for x in single for y in single for z in single]

def init_player(random_bits):
    players = [Player([r[0], r[1]]) for r in random_bits]
    return players[0], players[1], players[2]

random_bits_list = generate_randomness()
print(len(random_bits_list))

vic = Verifier()
optimal_strategy = []
for random_bits in random_bits_list:
    Alice, Bob, Charlie = init_player(random_bits)
    win_prob = 0
    for question in vic.questions:
        win_prob = win_prob + vic.judge(question,
                                        Alice.answer(vic.query_alice(question)),
                                        Bob.answer(vic.query_bob(question)),
                                        Charlie.answer(vic.query_charlie(question))
                                       )
    if win_prob == 3:
        optimal_strategy.append(random_bits)
print(len(optimal_strategy))

64
32


In [5]:
for s in optimal_strategy:
    print(s)

[[0, 0], [0, 0], [0, 1]]
[[0, 0], [0, 0], [1, 0]]
[[0, 0], [0, 1], [0, 0]]
[[0, 0], [0, 1], [1, 1]]
[[0, 0], [1, 0], [0, 0]]
[[0, 0], [1, 0], [1, 0]]
[[0, 0], [1, 1], [0, 1]]
[[0, 0], [1, 1], [1, 1]]
[[0, 1], [0, 0], [0, 0]]
[[0, 1], [0, 0], [1, 1]]
[[0, 1], [0, 1], [0, 1]]
[[0, 1], [0, 1], [1, 0]]
[[0, 1], [1, 0], [0, 1]]
[[0, 1], [1, 0], [1, 1]]
[[0, 1], [1, 1], [0, 0]]
[[0, 1], [1, 1], [1, 0]]
[[1, 0], [0, 0], [0, 0]]
[[1, 0], [0, 0], [1, 0]]
[[1, 0], [0, 1], [0, 1]]
[[1, 0], [0, 1], [1, 1]]
[[1, 0], [1, 0], [0, 0]]
[[1, 0], [1, 0], [1, 1]]
[[1, 0], [1, 1], [0, 1]]
[[1, 0], [1, 1], [1, 0]]
[[1, 1], [0, 0], [0, 1]]
[[1, 1], [0, 0], [1, 1]]
[[1, 1], [0, 1], [0, 0]]
[[1, 1], [0, 1], [1, 0]]
[[1, 1], [1, 0], [0, 1]]
[[1, 1], [1, 0], [1, 0]]
[[1, 1], [1, 1], [0, 0]]
[[1, 1], [1, 1], [1, 1]]


# How to interprete the strategies:

strategies where r_0 is 0 means x * 0 \xor r_1 = r_1, a constant output of r_1 regardless of the input bit x.

strategies where r_0 is 1 measn x * 1 \xor r_1 = x \xor r_1.

Therefore the following 4 strategies means:
* [0, 0] = 0
* [0, 1] = 1
* [1, 0] = x * 1 + 0 = x
* [1, 1] = x * 1 + 1 = \bar{x}

where x is the input bit.

## We will now compute the exclusion set

exclusion set is defined to be the set of inputs that an optimal deterministic strategy fails at.

In [10]:
exclusion = {}

for i in range(len(optimal_strategy)):
    Alice, Bob, Charlie = init_player(optimal_strategy[i])
    for question in vic.questions:
        if not vic.judge(question, 
                         Alice.answer(vic.query_alice(question)),
                         Bob.answer(vic.query_bob(question)),
                         Charlie.answer(vic.query_charlie(question))
                        ):
            exclusion['E{}'.format(i)] = (question, optimal_strategy[i])
print(exclusion)
    

{'E0': ([0, 0, 0], [[0, 0], [0, 0], [0, 1]]), 'E1': ([1, 1, 0], [[0, 0], [0, 0], [1, 0]]), 'E2': ([0, 0, 0], [[0, 0], [0, 1], [0, 0]]), 'E3': ([1, 1, 0], [[0, 0], [0, 1], [1, 1]]), 'E4': ([1, 0, 1], [[0, 0], [1, 0], [0, 0]]), 'E5': ([0, 1, 1], [[0, 0], [1, 0], [1, 0]]), 'E6': ([1, 0, 1], [[0, 0], [1, 1], [0, 1]]), 'E7': ([0, 1, 1], [[0, 0], [1, 1], [1, 1]]), 'E8': ([0, 0, 0], [[0, 1], [0, 0], [0, 0]]), 'E9': ([1, 1, 0], [[0, 1], [0, 0], [1, 1]]), 'E10': ([0, 0, 0], [[0, 1], [0, 1], [0, 1]]), 'E11': ([1, 1, 0], [[0, 1], [0, 1], [1, 0]]), 'E12': ([1, 0, 1], [[0, 1], [1, 0], [0, 1]]), 'E13': ([0, 1, 1], [[0, 1], [1, 0], [1, 1]]), 'E14': ([1, 0, 1], [[0, 1], [1, 1], [0, 0]]), 'E15': ([0, 1, 1], [[0, 1], [1, 1], [1, 0]]), 'E16': ([0, 1, 1], [[1, 0], [0, 0], [0, 0]]), 'E17': ([1, 0, 1], [[1, 0], [0, 0], [1, 0]]), 'E18': ([0, 1, 1], [[1, 0], [0, 1], [0, 1]]), 'E19': ([1, 0, 1], [[1, 0], [0, 1], [1, 1]]), 'E20': ([1, 1, 0], [[1, 0], [1, 0], [0, 0]]), 'E21': ([0, 0, 0], [[1, 0], [1, 0], [1, 1]]

From above, we can see that for the same input, not just one optimal strategy fails at it. This is also obvious due to that fact that we have only 4 possible input sets, and 32 optimal deterministic strategies.
We now use the equivalence relationship between exclusion sets to group them once more under some relabeling.

In [19]:
name_conversion = {'E0': [0, 0, 0], 'E1': [0, 1, 1], 'E2': [1, 0, 1], 'E3': [1, 1, 0]}
exclusion_sets = {'E0': [], 'E1': [], 'E2': [], 'E3': []}
i = 0
for (question, strategy) in exclusion.values():
    if question == [0, 0, 0]:
        exclusion_sets['E0'].append(strategy)
    elif question == [0, 1, 1]:
        exclusion_sets['E1'].append(strategy)
    elif question == [1, 0, 1]:
        exclusion_sets['E2'].append(strategy)
    else:
        exclusion_sets['E3'].append(strategy)

for key, value in exclusion_sets.items():
    print(key + ": ", name_conversion[key])
    for s in value:
        print(s)

    

E0:  [0, 0, 0]
[[0, 0], [0, 0], [0, 1]]
[[0, 0], [0, 1], [0, 0]]
[[0, 1], [0, 0], [0, 0]]
[[0, 1], [0, 1], [0, 1]]
[[1, 0], [1, 0], [1, 1]]
[[1, 0], [1, 1], [1, 0]]
[[1, 1], [1, 0], [1, 0]]
[[1, 1], [1, 1], [1, 1]]
E1:  [0, 1, 1]
[[0, 0], [1, 0], [1, 0]]
[[0, 0], [1, 1], [1, 1]]
[[0, 1], [1, 0], [1, 1]]
[[0, 1], [1, 1], [1, 0]]
[[1, 0], [0, 0], [0, 0]]
[[1, 0], [0, 1], [0, 1]]
[[1, 1], [0, 0], [0, 1]]
[[1, 1], [0, 1], [0, 0]]
E2:  [1, 0, 1]
[[0, 0], [1, 0], [0, 0]]
[[0, 0], [1, 1], [0, 1]]
[[0, 1], [1, 0], [0, 1]]
[[0, 1], [1, 1], [0, 0]]
[[1, 0], [0, 0], [1, 0]]
[[1, 0], [0, 1], [1, 1]]
[[1, 1], [0, 0], [1, 1]]
[[1, 1], [0, 1], [1, 0]]
E3:  [1, 1, 0]
[[0, 0], [0, 0], [1, 0]]
[[0, 0], [0, 1], [1, 1]]
[[0, 1], [0, 0], [1, 1]]
[[0, 1], [0, 1], [1, 0]]
[[1, 0], [1, 0], [0, 0]]
[[1, 0], [1, 1], [0, 1]]
[[1, 1], [1, 0], [0, 1]]
[[1, 1], [1, 1], [0, 0]]


In this case, we can see that for each input, there are exactly 8 strategies that fails at it. In other words, the exclusion sets for Mermin-GHZ game are symmetric.